# IMPORT LIBRARIES

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from env import user, password, host
from scipy import stats
from scipy.stats import levene, ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from datetime import datetime
import math
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import time
import warnings
warnings.filterwarnings("ignore")
# ?from pandas_profiling import ProfileReport

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

sns.set(color_codes=True)

from datetime import datetime

In [ ]:
%lsmagic

# Purpose of the this project
### I am trying to classify the city services request from the calls received.  The metric to be used is accuracy.  This will enable better customer service by providing useful data analysis to help determine funding and ataffing allocations to departments.

# ACQUIRE

In [ ]:
def get_311_data():
    ''' 
    This function reads in a csv held in the same repository folder
    '''
    df = pd.read_csv('data_311.csv')
    return df


In [ ]:
df=get_311_data()

# SUMMARIZE DATA 
#### WHAT DO WE HAVE HERE?

In [ ]:
# return first df rows
df.head()

#### Looks like a log of calls for city service departments

In [ ]:
# return last df rows
df.tail()

In [ ]:
#list column names
df.columns

#### Need to see if columns have REDUNDANCY and see if column will be useful 

In [ ]:
# give numerical data, stats, min, max, quartile info
df.describe()

In [ ]:
# list number columns and rows, data types,and non null values
df.info()

#### Most are object but case_id is an integer and num_days_late is a float. Also, I might want to organize it by the case_id, or by date, maybe by case_opened_date.

In [ ]:
# number null values in each column
df.isnull().sum()

In [ ]:
missing = df.isnull().sum()
missing[missing >0]

In [ ]:
18333/855269

#### But it isn't a large amount (only 2.1%), so I will consider dropping columns with nulls

#### NOTE for the cells below. I know that the following is a bit much, but I really need to see how the data is formatted

In [ ]:
# Use .unique and .nunique with object columns.

obj_cols = df.columns[[df[col].dtype == 'O' for col in df.columns]]
for col in obj_cols:
    print(df[col])
    print(df[col].unique()) #prints actual values in column
    print(df[col].nunique()) #prints number of unique values
    print('----------------------')


In [ ]:
# see period of time 
df.case_opened_date.min(), df.case_opened_date.max()


In [ ]:
# Use .value_counts() with object columns to get actual and percent values in categorical columns

obj_cols = df.columns[[df[col].dtype == 'O' for col in df.columns]]
for col in obj_cols:
    print(df[col].value_counts()) #prints actual count
    print(df[col].value_counts(normalize=True, dropna=False)) #prints %
    print('----------------------')


# PREP & CLEAN
#### drop duplicates, get rid of nulls, drop some columns, encode columns

In [ ]:
# Total number of rows and columns
df.shape

In [ ]:
# Rows containing duplicate data
duplicate_rows_df = df[df.duplicated()]
print('number of duplicate rows: ', duplicate_rows_df.shape)

In [ ]:
df = df.drop_duplicates()

In [ ]:
# Dropping the missing values.
df = df.dropna() 
df.count()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df = df[df['council_district'].str.contains('YES|Code Enforcement Services')==False]

In [ ]:
df['council_district'].unique()

In [ ]:
df['council_district'] = df['council_district'].astype(str)

In [ ]:
df['council_district'].unique()

In [2]:
late_mean=df['num_days_late'].mean()
late_mean

NameError: name 'df' is not defined

In [ ]:
df=df.drop(columns=['source_id', 'case_id', 'SLA_due_date', 'case_closed', 'SLA_days', 'case_status', 'dept_name',
                    'dept_subject_to_SLA', 'index','source_username'])

### Number of days late has both negative and positive values. Requests have to be completed in a timely manner, so if it is a negative number then it was completed before the due date, so I am going to encode 'case_late' column

In [ ]:
df['case_late_encoded'] = df.case_late.map({'YES': 1, 'NO':0})

In [ ]:
df.head()

# EDA

In [ ]:
def split_311_data(df):
    '''
    This function performs split on 311 data, stratify case_late_encoded
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.case_late_encoded)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.case_late_encoded)
    return train, validate, test

In [ ]:
train, validate, test= split_311_data(df)

In [ ]:
my_tab = pd.crosstab(index = train['case_late_encoded'],  # Make a crosstab
                              columns="count")      # Name the count column

my_tab.plot.bar()

In [ ]:
my_tab = pd.crosstab(index = train['standardized_dept_name'],  # Make a crosstab
                              columns="count")      # Name the count column

my_tab.plot.bar()

In [ ]:
my_tab = pd.crosstab(index = train['council_district'],  # Make a crosstab
                              columns="count")      # Name the count column

my_tab.plot.bar()

In [ ]:
my_tab = pd.crosstab(index = train['case_late'],  # Make a crosstab
                              columns="count")      # Name the count column

my_tab.plot.bar()

In [ ]:
my_tab = pd.crosstab(index = train['dept_division'],  # Make a crosstab
                              columns="count")      # Name the count column

my_tab.plot.bar()

In [ ]:
# Check out distributions of numeric columns.

num_cols = train.columns[[train[col].dtype == 'float64' for col in train.columns]]
for col in num_cols:
    plt.hist(train[col])
    plt.title(col)
    plt.show()

In [ ]:
#two-way table
grouped = train.groupby(['case_late','dept_division'])
grouped.size()

In [ ]:
table = pd.crosstab(index=train['case_late'], columns=train['service_request_type'])
table

In [ ]:
table.plot(kind='bar', figsize=(8,8), stacked=True)

In [ ]:
table = pd.crosstab(index=train['case_late'], columns=train['council_district'])
table

In [ ]:
table.plot(kind='bar', figsize=(8,8), stacked=True)

In [ ]:
table = pd.crosstab(index=train['case_late'], columns=train['dept_division'])

table

In [ ]:
table.plot(kind='bar', figsize=(8,8), stacked=True)

In [ ]:
plt.figure(figsize=(20, 20))
sns.violinplot(x='standardized_dept_name', y='case_late_encoded' , data=train)


In [ ]:
plt.figure(figsize=(20, 20))
sns.violinplot(x='standardized_dept_name', y='num_days_late' , data=train)

In [ ]:
plt.figure(figsize=(20, 20))
sns.boxplot(train['num_days_late'])

In [ ]:
plt.figure(figsize=(20, 20))
sns.barplot(train['num_days_late'], train['standardized_dept_name'])

In [ ]:
plt.figure(figsize=(20, 20))
sns.barplot(train['num_days_late'], train['council_district'])

In [ ]:
plt.figure(figsize=(20, 20))
sns.barplot(train['num_days_late'], train['dept_division'])

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1,1,1)
ax.scatter(train['standardized_dept_name'], train['num_days_late'])

plt.title('Days Late and Departments')
plt.xlabel('Department')
plt.ylabel('Days Late')
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1,1,1)
ax.scatter(train['council_district'], train['num_days_late'])

plt.title('Days Late and Council Districts')
plt.xlabel('Council District')
plt.ylabel('Days Late')
plt.show()

In [ ]:
#  get numerical data, stats, min, max, quartile info for each district
by_council = train.groupby(['council_district'])
by_council.describe()

In [ ]:
by_dept = train.groupby(['standardized_dept_name'])
by_dept.describe()

In [ ]:
by_request= train.groupby(['service_request_type'])
by_request.describe()

In [ ]:
train.head()

In [ ]:
train.columns


In [ ]:
a= sns.FacetGrid(train, col='standardized_dept_name', row='case_late_encoded')
a.map(plt.hist, 'num_days_late')

In [ ]:
a= sns.FacetGrid(train, col='council_district', row='case_late_encoded')
a.map(plt.hist, 'num_days_late')

In [ ]:
a= sns.FacetGrid(train, col='dept_division', row='case_late_encoded')
a.map(plt.hist, 'num_days_late')

In [ ]:
a= sns.FacetGrid(train, col='service_request_type', row='case_late_encoded')
a.map(plt.hist, 'num_days_late')

In [ ]:
a= sns.FacetGrid(train, col='standardized_dept_name', row='case_late_encoded')
a.map(plt.hist, 'num_days_late')

In [1]:
sns.scatterplot(x='num_days_late', y='dept_division', data=train, hue='case_late_encoded')

NameError: name 'sns' is not defined

In [ ]:
df.average=df['num_days_late'].mean()
